<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/object_classification/cat_and_dog/mobilenet_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNet 推論テスト

テスト用画像
https://drive.google.com/drive/u/0/folders/139S7L2uDqgHxtVeV6lzjCg57ZJzybZCJ


## 環境設定

In [0]:
from google.colab import drive 
drive.mount("/content/drive")
!ls

In [0]:
# TEST_IMAGE_CAT = "/content/test_cat.jpg"
# !wget "https://drive.google.com/uc?export=download&id=1tHsxCDGzQiZanluZQ1WM0u0LPvnr4mLa" -O "$TEST_IMAGE_CAT"
# TEST_IMAGE_DOG = "/content/test_dog.jpg"
# !wget "https://drive.google.com/uc?export=download&id=1oNYA8CPF07flKI9_H9nSlZPpAyo_dIb2" -O "$TEST_IMAGE_DOG"

# TEST_IMAGE = TEST_IMAGE_CAT

from google.colab import files
uploaded = files.upload()
TEST_IMAGE = next(iter(uploaded.keys()))

## 提供されているPre/Post processを使用

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

# Load model
mobile_base = tf.keras.applications.MobileNetV2()
mobile_base.summary()
model_n, model_h, model_w, model_c = mobile_base.input_shape

# Read image
img_org = cv2.imread(TEST_IMAGE)
cv2_imshow(img_org)

# Pre process
img = cv2.resize(img_org, (model_w, model_h))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = preprocess_input(img)
input_tensor = np.expand_dims(input_tensor, axis=0)

# Inference
scores = mobile_base.predict(input_tensor)

# Post process
results = decode_predictions(scores)

print(results[0])

## 自分でPre/Post process

In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Load model
mobile_base = tf.keras.applications.MobileNetV2()
mobile_base.summary()
model_n, model_h, model_w, model_c = mobile_base.input_shape

# Read image
img_org = cv2.imread(TEST_IMAGE)
cv2_imshow(img_org)

# Pre process
img = cv2.resize(img_org, (model_w, model_h))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = img / 128.
input_tensor -= 1.
input_tensor = input_tensor.astype(np.float32)
input_tensor = np.expand_dims(input_tensor, axis=0)

# Inference
output_tensor = mobile_base(inputs=input_tensor)

# Post process
result = np.argmax(output_tensor)
score = output_tensor[0][result].numpy()
print("predicted number is {} [{:.2f}]".format(result, score))

# 特徴マップを抽出する


In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Load model
mobile_base = tf.keras.applications.MobileNetV2()
mobile_base.summary()
model_n, model_h, model_w, model_c = mobile_base.input_shape

# Modify model to get internal layer
model = tf.keras.models.Model(inputs=mobile_base.input, outputs=mobile_base.get_layer("block_1_project").output)

# Read image
img_org = cv2.imread(TEST_IMAGE)
cv2_imshow(img_org)

# Pre process
img = cv2.resize(img_org, (model_w, model_h))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_tensor = preprocess_input(img)
input_tensor = np.expand_dims(input_tensor, axis=0)

# Inference
feature_map = model(input_tensor)

# Display feature map
num = feature_map.shape[3]
plt.figure(figsize=(20, 20))
for i in range(num):
  plt.subplot((int)(num/8) , 8, i + 1)
  plt.imshow(feature_map[0,:,:,i])